In [5]:
import splot
import geopandas as gpd
import pandas as pd
import numpy as np
from libpysal.weights.spatial_lag import lag_spatial as slag
from libpysal.weights.contiguity import Queen

# Assume the adata1 is the observed gene expression, and adata2 is the predicted gene expression
adata1 = pd.DataFrame(np.random.rand(4,4))
adata2 = pd.DataFrame(np.random.rand(4,4))

# Assume the coords is the spatial coordinates of the spots
coords = np.array([[0,2],[2,3],[1,2],[4,2]])

# Assume the x and y are the observed and predicted gene expression
x = adata1.iloc[0, :]
y = adata2.iloc[0, :]

# This is just compute geometry from coordinates
gpd_adata = gpd.GeoDataFrame(adata1, geometry=gpd.points_from_xy(coords[:,0], coords[:,1]))
gpd_adata


,0,1,2,3,geometry
0,0.795022,0.917594,0.739225,0.914600,POINT (0.00000 2.00000)
1,0.892427,0.846817,0.955910,0.188620,POINT (2.00000 3.00000)
2,0.042565,0.823697,0.551146,0.436727,POINT (1.00000 2.00000)
3,0.622213,0.560312,0.157464,0.576156,POINT (4.00000 2.00000)


In [6]:
# Here is how to compute the weights
w = Queen.from_dataframe(gpd_adata)
print("original weight matrix:\n", w.sparse.toarray())
print(w.weights)
print()
w.transform = "r"

# We can see the weights
print("transformed weight matrix:\n", w.sparse.toarray())
print(w.weights)


original weight matrix:
 [[0. 0. 1. 0.]
 [0. 0. 1. 1.]
 [1. 1. 0. 0.]
 [0. 1. 0. 0.]]
{0: [1.0], 1: [1.0, 1.0], 2: [1.0, 1.0], 3: [1.0]}

transformed weight matrix:
 [[0.  0.  1.  0. ]
 [0.  0.  0.5 0.5]
 [0.5 0.5 0.  0. ]
 [0.  1.  0.  0. ]]
{0: [1.0], 1: [0.5, 0.5], 2: [0.5, 0.5], 3: [1.0]}


/tmp/ipykernel_481433/1628162136.py:2: FutureWarning: `use_index` defaults to False but will default to True in future. Set True/False directly to control this behavior and silence this warning
  w = Queen.from_dataframe(gpd_adata)


In [7]:
# z-score
zy = (y - y.mean()) / y.std(ddof=1)
zx = (x - x.mean()) / x.std(ddof=1)

n = x.shape[0]
den = n - 1.0 
wzy = w.sparse * zy
num = (zx * wzy.T).sum()
Moran_I = num / den
Moran_I


0.018689501898518052